In [1]:
import json
import pytorch_lightning as pl



from model_AB import *
from model_CD import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PAD_TOKEN = '<pad>'
EN_TRAIN_PATH = "./../../data/EN/train.json"
EN_DEV_PATH = "./../../data/EN/dev.json"
BERT_PATH = "./../../model/bert-base-cased"
#BERT_PATH = "./model/bert-base-cased"
VERBATLAS_PATH = "./VerbAtlas/VerbAtlas"

cuda
11.3
verbatlas scraped
cuda
11.3


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/michele/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /home/michele/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/michele/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
def read_dataset(path: str):
    with open(path) as f:
        dataset = json.load(f)

    sentences, labels = {}, {}
    for sentence_id, sentence in dataset.items():
        sentence_id = sentence_id
        sentences[sentence_id] = {
            "words": sentence["words"],
            "lemmas": sentence["lemmas"],
            "pos_tags": sentence["pos_tags"],
            "dependency_heads": [int(head) for head in sentence["dependency_heads"]],
            "dependency_relations": sentence["dependency_relations"],
            "predicates": sentence["predicates"],
        }

        labels[sentence_id] = {
            "predicates": sentence["predicates"],
            "roles": {int(p): r for p, r in sentence["roles"].items()}
            if "roles" in sentence
            else dict(),
        }

    return sentences, labels

In [ ]:
#'''
early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_f1',  # the value that will be evaluated to activate the early stopping of the model.
    patience=5,
    # the number of consecutive attempts that the model has to raise (or lower depending on the metric used) to raise the "monitor" value.
    verbose=True,  # whether to log or not information in the console.
    mode='max',  # wheter we want to maximize (max) or minimize the "monitor" value.
)

check_point_callback = pl.callbacks.ModelCheckpoint(
    monitor='val_f1',  # the value that we want to use for model selection.
    verbose=True,  # whether to log or not information in the console.
    save_top_k=1,  # the number of checkpoints we want to store.
    mode='max',  # wheter we want to maximize (max) or minimize the "monitor" value.
    filename='modelAB_{epoch}-{val_f1:.4f}'
    # the prefix on the checkpoint values. Metrics store by the trainer can be used to dynamically change the name.
)

sentences_dm = PredicatesDataModule(
    data_train_path=EN_TRAIN_PATH,
    data_dev_path=EN_DEV_PATH,
    data_test_path=EN_DEV_PATH,
    batch_size=32
)

classifier = AB_Model(num_classes=len(frame2id)+2,language="en", hidden1=768, lstm_layers=1, bidirectional=False, p=0.5).to(device)

# the PyTorch Lightning Trainer
trainer = pl.Trainer(
    max_epochs=10,  # maximum number of epochs.
    gpus=1,  # the number of gpus we have at our disposal.
    callbacks=[early_stopping, check_point_callback]  # the callback we want our trainer to use.
)

# and finally we can let the "trainer" fit the amazon reviews classifier.
trainer.fit(model=classifier, datamodule=sentences_dm)

model_path = "../../model/modelAB.ckpt"
#classifier = AB_Model.load_from_checkpoint(model_path,num_classes=len(frame2id)+2 , language="en").to(device)




sentences, labels = read_dataset(EN_DEV_PATH)

for idx, key in enumerate(sentences):
    prediction = classifier.predict(sentences[key])
    lab = labels[key]["predicates"]
    print("PREDICTED", len(prediction), prediction)
    print("GROUND TRUTH", len(lab), lab)

NUM_CLASSES  434


Some weights of the model checkpoint at ./../../model/bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ./../../model/bert-base-cased and are newly initialized: ['bert.enc

Validation sanity check: 0it [00:00, ?it/s]

/home/michele/anaconda3/envs/nlp2022-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/michele/anaconda3/envs/nlp2022-hw2/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:59: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/michele/anaconda3/envs/nlp2022-hw2/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 whic

Training: 0it [00:00, ?it/s]

In [ ]:
#'''
early_stopping = pl.callbacks.EarlyStopping(
    monitor='val_f1',  # the value that will be evaluated to activate the early stopping of the model.
    patience=5,
    # the number of consecutive attempts that the model has to raise (or lower depending on the metric used) to raise the "monitor" value.
    verbose=True,  # whether to log or not information in the console.
    mode='max',  # wheter we want to maximize (max) or minimize the "monitor" value.
)

check_point_callback = pl.callbacks.ModelCheckpoint(
    monitor='val_f1',  # the value that we want to use for model selection.
    verbose=True,  # whether to log or not information in the console.
    save_top_k=1,  # the number of checkpoints we want to store.
    mode='max',  # wheter we want to maximize (max) or minimize the "monitor" value.
    filename='modelAB_{epoch}-{val_f1:.4f}'
    # the prefix on the checkpoint values. Metrics store by the trainer can be used to dynamically change the name.
)

sentences_dm = PredicatesDataModule(
    data_train_path=EN_TRAIN_PATH,
    data_dev_path=EN_DEV_PATH,
    data_test_path=EN_DEV_PATH,
    batch_size=32
)

classifier = AB_Model(num_classes=len(frame2id)+2,language="en", hidden1=768, lstm_layers=1, bidirectional=False, p=0.).to(device)

# the PyTorch Lightning Trainer
trainer = pl.Trainer(
    max_epochs=1,  # maximum number of epochs.
    gpus=1,  # the number of gpus we have at our disposal.
    callbacks=[early_stopping, check_point_callback]  # the callback we want our trainer to use.
)

# and finally we can let the "trainer" fit the amazon reviews classifier.
trainer.fit(model=classifier, datamodule=sentences_dm)

model_path = "../../model/modelAB.ckpt"
#classifier = AB_Model.load_from_checkpoint(model_path,num_classes=len(frame2id)+2 , language="en").to(device)





sentences, labels = read_dataset(EN_DEV_PATH)

for idx, key in enumerate(sentences):
    prediction = classifier.predict(sentences[key])
    lab = labels[key]["predicates"]
    print("PREDICTED", len(prediction), prediction)
    print("GROUND TRUTH", len(lab), lab)
#'''